Подготовка необходимых библиотек

In [ ]:
import subprocess
import geopandas as gpd
import pandas as pd

def install_packages(requirements_file): 
    print(f"Установлены: {', '.join([line.split()[1] for line in subprocess.run(['pip', 'install', '--upgrade', '-r', requirements_file], capture_output=True, text=True).stdout.splitlines() if 'Successfully installed' in line])}" if 'Successfully installed' in subprocess.run(['pip', 'install', '--upgrade', '-r', requirements_file], capture_output=True, text=True).stdout else "Пакеты уже актуальны.")

# Пример использования
install_packages('requirements/requirements.txt')
install_packages('requirements/dev-requirements.txt')

Подготовка слоёв, проверка необходимых атрибутов

In [ ]:
from check_geojson import check_geojson
import json

# Папка, где лежат required_columns.json и слои
folder = "path_to_folder"

# Функция для загрузки кодов жилых зон из JSON файла
def load_living_codes(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        return set(data.get("living_codes", []))  # Возвращаем множество кодов жилых зон
    except Exception as e:
        print(f"Ошибка при загрузке living_codes: {e}")
        return set()

# Вызываем проверку и загрузку данных
data = check_geojson(folder)

# Достаём слои
if data:
    buildings   = data.get("buildings")
    zones       = data.get("zones")
    school      = data.get("school")
    polyclinic  = data.get("polyclinic")
    kindergarten= data.get("kindergarten")
    green       = data.get("green")
    park        = data.get("park")

    # Загружаем living_codes из файла
    living_codes_path = "living_codes.json"
    living_codes = load_living_codes(living_codes_path)

    # Посмотрим, что загружено
    print("Загруженные слои:")
    for name, gdf in data.items():
        print(f"  - {name}: {len(gdf)} объектов")

    # Дополнительно выводим коды жилых зон
    print(f"Загруженные коды жилых зон: {living_codes}")
else:
    print("Слои не загружены.")

Присвоение типа модели городской застройки окружающим зонам на территории которых возможно размщение социальных сервисов

Расчёт физических параметров застройки в территориальных зонах, подсчёт проживающего населения - city_model_processing

In [ ]:
# Импортируем необходимые библиотеки
import geopandas as gpd
import pandas as pd
from city_model_processing import *
from apply_styles import apply_styles, create_map

# Указываем информацию о населении в исследуемом городе
living_population = # вставьте свое число жителей в городе

# Производим присвоение новых атрибутов зданиям и территориальным зонам
zones, balanced_buildings = process_city_model(zones, buildings, living_population, living_codes_path)

# Визуализация
zones = apply_styles(zones)
m_1 = create_map(zones)
m_1

Проставление атрибутов (радиуса доступности по городским моделям, при необходимости вместимости сервисов) с помощью service_data_processing

Расчёт вакантных/занятых мест в социальных сервисах с помощью calculating_provision

Определение участков, которые входят в радиус доступности социальных сервисов, присвоение атрибутов свободных/вакантных мест территориальным зонам в радиусе доступности - social_infrastructure_mapper 

In [ ]:
from shapely.ops import unary_union
from objectnat import get_adj_matrix_gdf_to_gdf, get_intermodal_graph
 
# Загружаем геометрии
services = gpd.read_file(r"path_to_file") # необходимо указать путь до любого сервиса или границу города
boundary = gpd.read_file(r"path_to_file")

# Фильтрация пустых и невалидных геометрий
balanced_buildings = balanced_buildings[~balanced_buildings.geometry.is_empty & balanced_buildings.geometry.notnull() & balanced_buildings.is_valid]
services = services[~services.geometry.is_empty & services.geometry.notnull() & services.is_valid]
# Фильтрация пустых и невалидных геометрий
balanced_buildings = balanced_buildings[~balanced_buildings.geometry.is_empty & balanced_buildings.geometry.notna() & balanced_buildings.is_valid]
services = services[~services.geometry.is_empty & services.geometry.notna() & services.is_valid]

# buildings = gpd.read_file(r"ТВОЙ_ПУТЬ_К_ЗДАНИЯМ.geojson")  # если ещё не загружены

# Объединяем все геометрии
all_geoms = balanced_buildings.to_crs(4326).geometry.tolist() + services.to_crs(4326).geometry.tolist()
union_geom = unary_union(all_geoms)

# Строим буферную оболочку
polygon = union_geom.convex_hull.buffer(0.001)
gdf = gdf[~gdf.geometry.is_empty & gdf.geometry.notna()]
valid_geometry = ~gdf.geometry.is_empty & gdf.geometry.notna()
gdf = gdf[valid_geometry]

# Строим граф
G_intermodal = get_intermodal_graph(polygon=polygon, clip_by_bounds=True)

# Строим матрицу смежности
matrix: pd.DataFrame = get_adj_matrix_gdf_to_gdf(
    gdf_from=balanced_buildings,
    gdf_to=services,
    nx_graph=G_intermodal,
    weight="time_min",
    threshold=45
)


In [ ]:
import pandas as pd
matrix.to_csv(r"path_to_file")

In [ ]:
# Присваиваем атрибуты доступности, вместимости, параметры здания сервиса
from service_data_processing import process_service_data, SERVICE_TYPES, BUFFER_SIZES

combined_service = process_service_data(school, kindergarten, polyclinic, balanced_buildings)

In [ ]:
# Вычисление обеспеченности сервисами

from calculating_provision import process_services
import pandas as pd

balanced_buildings.index = balanced_buildings.index.astype(int)
matrix.index = matrix.index.astype(int)
matrix.columns = matrix.columns.astype(int)


school, kindergarten, polyclinic = process_services(matrix, combined_service, balanced_buildings, output_path=r"D:\STUDY\ВКР\VKR_CODE\result_3")

from social_infrastructure_mapper import process_services

living_zones = process_services(zones, kindergarten, school, polyclinic)


In [ ]:
living_zones.to_file(r"path_to_file") #сохраните для второго этапа 
zones.to_file(r"path_to_file") #сохраните для второго этапа 